# Проект 9 — Самоорганизующиеся карты (Self-Organizing Maps, SOM)

## Теоретическая часть (Theoretical Background)

Самоорганизующиеся карты (Self-Organizing Maps, SOM) — это нейронные сети без учителя, предложенные Тево Кохоненом, для отображения многомерных данных на двумерную решётку.

SOM позволяет визуализировать сложные структуры данных и выявлять кластеры.

### Основные идеи:

1. **Архитектура:**
   - Сеть состоит из нейронов, расположенных в сетке (обычно 2D)
   - Каждый нейрон имеет весовой вектор той же размерности, что и входные данные

2. **Обучение:**
   - На каждом шаге выбирается входной вектор, и определяется нейрон-победитель (Best Matching Unit, BMU)
   - Веса нейрона-победителя и его соседей обновляются
   - Функция соседства уменьшается с расстоянием

3. **Свойства:**
   - Сохраняет топологию данных
   - Уменьшает размерность
   - Обеспечивает визуальное представление кластеров

4. **Типичные применения:**
   - Кластеризация данных
   - Визуализация многомерных признаков
   - Сегментация клиентов, изображений или биологических данных


## Импорт библиотек (Import Libraries)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import kagglehub
import os
import warnings
warnings.filterwarnings('ignore')

# Настройка для отображения графиков
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    try:
        plt.style.use('seaborn-darkgrid')
    except:
        plt.style.use('ggplot')
sns.set_palette("husl")
%matplotlib inline


## Задача 1: Подготовка данных (Data Preparation)

Загрузим и подготовим данные Customer Personality Analysis Dataset.


In [ ]:
# Загрузка датасета Customer Personality Analysis с Kaggle
print("Загрузка датасета с Kaggle...")
try:
    # Download latest version
    path = kagglehub.dataset_download("imakash3011/customer-personality-analysis")
    print("Path to dataset files:", path)
    
    # Ищем файл с данными в загруженной директории
    # Обычно файл называется marketing_campaign.csv
    csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
    
    if csv_files:
        # Используем первый найденный CSV файл
        csv_file = os.path.join(path, csv_files[0])
        print(f"Найден файл: {csv_file}")
        
        # Пробуем загрузить с разными разделителями
        try:
            df = pd.read_csv(csv_file, sep='\t')
            print("Данные загружены с разделителем табуляции")
        except:
            try:
                df = pd.read_csv(csv_file, sep=',')
                print("Данные загружены с разделителем запятой")
            except:
                df = pd.read_csv(csv_file)
                print("Данные загружены с автоматическим определением разделителя")
    else:
        # Если CSV файл не найден, ищем в поддиректориях
        for root, dirs, files in os.walk(path):
            for file in files:
                if file.endswith('.csv'):
                    csv_file = os.path.join(root, file)
                    print(f"Найден файл в поддиректории: {csv_file}")
                    try:
                        df = pd.read_csv(csv_file, sep='\t')
                        print("Данные загружены с разделителем табуляции")
                        break
                    except:
                        try:
                            df = pd.read_csv(csv_file, sep=',')
                            print("Данные загружены с разделителем запятой")
                            break
                        except:
                            df = pd.read_csv(csv_file)
                            print("Данные загружены с автоматическим определением разделителя")
                            break
            else:
                continue
            break
        else:
            raise FileNotFoundError("CSV файл не найден в загруженном датасете")
            
except Exception as e:
    print(f"Ошибка при загрузке с Kaggle: {e}")
    print("Попытка загрузить из локального файла...")
    try:
        df = pd.read_csv('marketing_campaign.csv', sep='\t')
        print("Данные загружены из локального файла")
    except FileNotFoundError:
        print("Локальный файл не найден. Используются синтетические данные для демонстрации...")
        # Создаём синтетические данные для демонстрации
        np.random.seed(42)
        n_samples = 2000
        df = pd.DataFrame({
            'Year_Birth': np.random.randint(1940, 2000, n_samples),
            'Education': np.random.choice(['Graduation', 'PhD', 'Master', 'Basic', '2n Cycle'], n_samples),
            'Marital_Status': np.random.choice(['Single', 'Together', 'Married', 'Divorced', 'Widow'], n_samples),
            'Income': np.random.normal(50000, 20000, n_samples),
            'Kidhome': np.random.choice([0, 1, 2], n_samples),
            'Teenhome': np.random.choice([0, 1, 2], n_samples),
            'Recency': np.random.randint(0, 100, n_samples),
            'MntWines': np.random.poisson(300, n_samples),
            'MntFruits': np.random.poisson(50, n_samples),
            'MntMeatProducts': np.random.poisson(200, n_samples),
            'MntFishProducts': np.random.poisson(50, n_samples),
            'MntSweetProducts': np.random.poisson(50, n_samples),
            'MntGoldProds': np.random.poisson(50, n_samples),
            'NumDealsPurchases': np.random.poisson(2, n_samples),
            'NumWebPurchases': np.random.poisson(5, n_samples),
            'NumCatalogPurchases': np.random.poisson(3, n_samples),
            'NumStorePurchases': np.random.poisson(6, n_samples),
            'NumWebVisitsMonth': np.random.poisson(5, n_samples),
            'AcceptedCmp1': np.random.choice([0, 1], n_samples),
            'AcceptedCmp2': np.random.choice([0, 1], n_samples),
            'AcceptedCmp3': np.random.choice([0, 1], n_samples),
            'AcceptedCmp4': np.random.choice([0, 1], n_samples),
            'AcceptedCmp5': np.random.choice([0, 1], n_samples),
            'Response': np.random.choice([0, 1], n_samples),
            'Complain': np.random.choice([0, 1], n_samples)
        })
        df['Income'] = np.abs(df['Income'])  # Убедимся, что доход положительный
        print("Синтетические данные созданы")

print(f"\nРазмер датасета: {df.shape}")
print(f"\nПервые строки:")
df.head()


In [ ]:
# Информация о данных
print("Информация о датасете:")
print(df.info())
print("\nОписательная статистика:")
df.describe()


In [ ]:
# Обработка пропущенных значений
print("Пропущенные значения:")
print(df.isnull().sum().sum())
if df.isnull().sum().sum() > 0:
    df = df.dropna()
    print(f"Удалены строки с пропущенными значениями. Новый размер: {df.shape}")

# Выбор ключевых признаков для анализа
# Выберем числовые признаки, связанные с поведением клиентов
feature_columns = [
    'Year_Birth',
    'Income',
    'Recency',
    'MntWines',
    'MntFruits',
    'MntMeatProducts',
    'MntFishProducts',
    'MntSweetProducts',
    'MntGoldProds',
    'NumDealsPurchases',
    'NumWebPurchases',
    'NumCatalogPurchases',
    'NumStorePurchases',
    'NumWebVisitsMonth'
]

# Проверяем наличие всех колонок
available_features = [col for col in feature_columns if col in df.columns]
print(f"\nИспользуемые признаки: {available_features}")

# Создаём датасет с выбранными признаками
data = df[available_features].copy()

# Вычисляем возраст из года рождения
if 'Year_Birth' in data.columns:
    data['Age'] = 2024 - data['Year_Birth']
    data = data.drop('Year_Birth', axis=1)

print(f"\nФинальный размер данных: {data.shape}")
print(f"\nПризнаки: {list(data.columns)}")
data.head()


In [ ]:
# Нормализация данных
# Импортируем необходимые библиотеки (на случай, если ячейка с импортами не была выполнена)
from sklearn.preprocessing import StandardScaler
import pandas as pd

scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)
data_scaled = pd.DataFrame(data_scaled, columns=data.columns)

print("Данные нормализованы (StandardScaler)")
print(f"Среднее: {data_scaled.mean().mean():.6f}")
print(f"Стандартное отклонение: {data_scaled.std().mean():.6f}")
print("\nПервые строки нормализованных данных:")
data_scaled.head()


## Задача 2: Реализация SOM (SOM Implementation)

Реализуем класс Self-Organizing Map с сеткой нейронов 10×10.


In [ ]:
class SelfOrganizingMap:
    """
    Реализация самоорганизующейся карты Кохонена (SOM)
    """
    
    def __init__(self, grid_size=(10, 10), input_dim=None, learning_rate=0.5, 
                 radius=None, random_seed=42):
        """
        Параметры:
        - grid_size: размер сетки (высота, ширина)
        - input_dim: размерность входных данных
        - learning_rate: начальная скорость обучения
        - radius: начальный радиус соседства (если None, вычисляется автоматически)
        - random_seed: seed для воспроизводимости
        """
        self.grid_size = grid_size
        self.input_dim = input_dim
        self.learning_rate = learning_rate
        self.random_seed = random_seed
        
        if radius is None:
            # Начальный радиус обычно равен половине размера сетки
            self.radius = max(grid_size) / 2
        else:
            self.radius = radius
        
        # Инициализация весов случайными значениями
        np.random.seed(random_seed)
        self.weights = np.random.randn(grid_size[0], grid_size[1], input_dim)
        
        # Создаём координаты нейронов в сетке
        self.coords = np.array([[i, j] for i in range(grid_size[0]) 
                                for j in range(grid_size[1])])
        self.coords = self.coords.reshape(grid_size[0], grid_size[1], 2)
        
    def find_bmu(self, input_vector):
        """
        Находит Best Matching Unit (BMU) - нейрон-победитель
        для данного входного вектора
        """
        # Вычисляем евклидово расстояние от входного вектора до всех нейронов
        distances = np.sum((self.weights - input_vector) ** 2, axis=2)
        # Находим индекс минимального расстояния
        bmu_idx = np.unravel_index(np.argmin(distances), distances.shape)
        return bmu_idx
    
    def neighborhood_function(self, distance, radius):
        """
        Функция соседства (гауссова)
        """
        return np.exp(-(distance ** 2) / (2 * (radius ** 2)))
    
    def update_weights(self, input_vector, bmu_idx, learning_rate, radius):
        """
        Обновляет веса нейронов на основе BMU и функции соседства
        """
        # Вычисляем расстояния от BMU до всех нейронов в сетке
        bmu_coords = np.array(bmu_idx)
        distances = np.sum((self.coords - bmu_coords) ** 2, axis=2)
        distances = np.sqrt(distances)
        
        # Вычисляем функцию соседства
        neighborhood = self.neighborhood_function(distances, radius)
        
        # Обновляем веса
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                influence = neighborhood[i, j] * learning_rate
                self.weights[i, j] += influence * (input_vector - self.weights[i, j])
    
    def train(self, data, n_iterations=1000, verbose=True):
        """
        Обучает SOM на данных
        
        Параметры:
        - data: нормализованные данные (numpy array или pandas DataFrame)
        - n_iterations: количество итераций обучения
        - verbose: выводить ли прогресс
        """
        if isinstance(data, pd.DataFrame):
            data = data.values
        
        n_samples = len(data)
        
        # Параметры для адаптивного обучения
        initial_lr = self.learning_rate
        initial_radius = self.radius
        
        for iteration in range(n_iterations):
            # Выбираем случайный образец
            sample_idx = np.random.randint(0, n_samples)
            sample = data[sample_idx]
            
            # Находим BMU
            bmu_idx = self.find_bmu(sample)
            
            # Адаптивно уменьшаем скорость обучения и радиус
            current_lr = initial_lr * np.exp(-iteration / n_iterations)
            current_radius = initial_radius * np.exp(-iteration / n_iterations)
            
            # Обновляем веса
            self.update_weights(sample, bmu_idx, current_lr, current_radius)
            
            if verbose and (iteration + 1) % (n_iterations // 10) == 0:
                print(f"Итерация {iteration + 1}/{n_iterations} завершена")
    
    def predict(self, data):
        """
        Предсказывает BMU для каждого образца данных
        """
        if isinstance(data, pd.DataFrame):
            data = data.values
        
        predictions = []
        for sample in data:
            bmu_idx = self.find_bmu(sample)
            predictions.append(bmu_idx)
        
        return np.array(predictions)
    
    def get_u_matrix(self):
        """
        Вычисляет U-Matrix (Unified Distance Matrix) для визуализации
        U-Matrix показывает средние расстояния между соседними нейронами
        """
        u_matrix = np.zeros(self.grid_size)
        
        for i in range(self.grid_size[0]):
            for j in range(self.grid_size[1]):
                neighbors = []
                
                # Проверяем соседей (вверх, вниз, влево, вправо)
                if i > 0:
                    neighbors.append(self.weights[i-1, j])
                if i < self.grid_size[0] - 1:
                    neighbors.append(self.weights[i+1, j])
                if j > 0:
                    neighbors.append(self.weights[i, j-1])
                if j < self.grid_size[1] - 1:
                    neighbors.append(self.weights[i, j+1])
                
                if neighbors:
                    # Вычисляем среднее расстояние до соседей
                    distances = [np.linalg.norm(self.weights[i, j] - neighbor) 
                                for neighbor in neighbors]
                    u_matrix[i, j] = np.mean(distances)
        
        return u_matrix


In [ ]:
# Создаём и обучаем SOM
print("Создание SOM с сеткой 10x10...")
som = SelfOrganizingMap(
    grid_size=(10, 10),
    input_dim=data_scaled.shape[1],
    learning_rate=0.5,
    random_seed=42
)

print("Начало обучения...")
som.train(data_scaled, n_iterations=1000, verbose=True)
print("Обучение завершено!")


In [ ]:
# Вычисляем U-Matrix
u_matrix = som.get_u_matrix()

# Визуализация U-Matrix
plt.figure(figsize=(12, 10))
plt.imshow(u_matrix, cmap='viridis', interpolation='nearest')
plt.colorbar(label='Среднее расстояние до соседей')
plt.title('U-Matrix (Unified Distance Matrix)', fontsize=16, fontweight='bold')
plt.xlabel('Ширина сетки')
plt.ylabel('Высота сетки')
plt.tight_layout()
plt.show()

# Интерпретация: тёмные области = кластеры, светлые = границы между кластерами


In [ ]:
# Альтернативная визуализация U-Matrix с контурными линиями
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Тепловая карта
im1 = ax1.imshow(u_matrix, cmap='viridis', interpolation='nearest')
ax1.set_title('U-Matrix (тепловая карта)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Ширина сетки')
ax1.set_ylabel('Высота сетки')
plt.colorbar(im1, ax=ax1, label='Среднее расстояние')

# Контурная карта
contour = ax2.contour(u_matrix, levels=15, cmap='viridis')
ax2.clabel(contour, inline=True, fontsize=8)
ax2.set_title('U-Matrix (контурная карта)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Ширина сетки')
ax2.set_ylabel('Высота сетки')

plt.tight_layout()
plt.show()


In [ ]:
# Визуализация распределения данных по карте
predictions = som.predict(data_scaled)

# Создаём карту частоты активации нейронов
activation_map = np.zeros(som.grid_size)
for pred in predictions:
    activation_map[pred[0], pred[1]] += 1

plt.figure(figsize=(12, 10))
plt.imshow(activation_map, cmap='YlOrRd', interpolation='nearest')
plt.colorbar(label='Количество активированных образцов')
plt.title('Карта активации нейронов', fontsize=16, fontweight='bold')
plt.xlabel('Ширина сетки')
plt.ylabel('Высота сетки')

# Добавляем аннотации с количеством
for i in range(som.grid_size[0]):
    for j in range(som.grid_size[1]):
        if activation_map[i, j] > 0:
            plt.text(j, i, int(activation_map[i, j]), 
                    ha='center', va='center', 
                    color='white' if activation_map[i, j] > activation_map.max()/2 else 'black',
                    fontweight='bold')

plt.tight_layout()
plt.show()


## Задача 4: Определение кластеров и интерпретация результатов

Определим кластеры клиентов на основе SOM и проанализируем их характеристики.


In [ ]:
# Используем K-Means на весах SOM для определения кластеров
# Это позволяет автоматически определить количество кластеров

# Получаем веса всех нейронов
neuron_weights = som.weights.reshape(-1, som.input_dim)

# Определяем оптимальное количество кластеров с помощью метода локтя
inertias = []
K_range = range(2, 11)
for k in K_range:
    kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans_temp.fit(neuron_weights)
    inertias.append(kmeans_temp.inertia_)

# Визуализация метода локтя
plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias, 'bo-')
plt.xlabel('Количество кластеров (K)')
plt.ylabel('Инерция (Inertia)')
plt.title('Метод локтя для определения оптимального K', fontsize=14, fontweight='bold')
plt.grid(True)
plt.tight_layout()
plt.show()

# Выбираем оптимальное K (обычно 4-6 для такого датасета)
optimal_k = 5
print(f"Выбрано количество кластеров: {optimal_k}")


In [ ]:
# Применяем K-Means к весам нейронов SOM
kmeans_som = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
neuron_clusters = kmeans_som.fit_predict(neuron_weights)
neuron_clusters = neuron_clusters.reshape(som.grid_size)

# Визуализация кластеров на карте
plt.figure(figsize=(12, 10))
plt.imshow(neuron_clusters, cmap='tab10', interpolation='nearest')
plt.colorbar(label='Кластер')
plt.title(f'Кластеризация SOM (K={optimal_k})', fontsize=16, fontweight='bold')
plt.xlabel('Ширина сетки')
plt.ylabel('Высота сетки')

# Добавляем границы кластеров
for i in range(som.grid_size[0]):
    for j in range(som.grid_size[1]):
        # Проверяем соседей для определения границ
        if i < som.grid_size[0] - 1 and neuron_clusters[i, j] != neuron_clusters[i+1, j]:
            plt.axhline(i + 0.5, color='black', linewidth=0.5)
        if j < som.grid_size[1] - 1 and neuron_clusters[i, j] != neuron_clusters[i, j+1]:
            plt.axvline(j + 0.5, color='black', linewidth=0.5)

plt.tight_layout()
plt.show()


In [ ]:
# Назначаем кластеры каждому образцу данных
data_predictions = som.predict(data_scaled)
sample_clusters = np.array([neuron_clusters[pred[0], pred[1]] for pred in data_predictions])

# Добавляем информацию о кластерах к исходным данным
df_with_clusters = df.copy()
df_with_clusters['SOM_Cluster'] = sample_clusters

print("Распределение образцов по кластерам:")
print(df_with_clusters['SOM_Cluster'].value_counts().sort_index())
print(f"\nВсего кластеров: {len(df_with_clusters['SOM_Cluster'].unique())}")


In [ ]:
# Анализ характеристик кластеров
print("=" * 80)
print("АНАЛИЗ ХАРАКТЕРИСТИК КЛАСТЕРОВ")
print("=" * 80)

cluster_analysis = df_with_clusters.groupby('SOM_Cluster')[available_features].mean()

# Добавляем размер кластера
cluster_sizes = df_with_clusters['SOM_Cluster'].value_counts().sort_index()
cluster_analysis['Cluster_Size'] = cluster_sizes.values

print("\nСредние значения признаков по кластерам:")
print(cluster_analysis.round(2))

# Визуализация характеристик кластеров
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Размер кластеров
axes[0, 0].bar(cluster_sizes.index, cluster_sizes.values, color='steelblue')
axes[0, 0].set_title('Размер кластеров', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Кластер')
axes[0, 0].set_ylabel('Количество клиентов')
axes[0, 0].grid(axis='y', alpha=0.3)

# 2. Доход по кластерам
if 'Income' in available_features:
    income_by_cluster = df_with_clusters.groupby('SOM_Cluster')['Income'].mean()
    axes[0, 1].bar(income_by_cluster.index, income_by_cluster.values, color='coral')
    axes[0, 1].set_title('Средний доход по кластерам', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('Кластер')
    axes[0, 1].set_ylabel('Средний доход')
    axes[0, 1].grid(axis='y', alpha=0.3)

# 3. Расходы на вино по кластерам
if 'MntWines' in available_features:
    wine_by_cluster = df_with_clusters.groupby('SOM_Cluster')['MntWines'].mean()
    axes[1, 0].bar(wine_by_cluster.index, wine_by_cluster.values, color='green')
    axes[1, 0].set_title('Средние расходы на вино по кластерам', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('Кластер')
    axes[1, 0].set_ylabel('Средние расходы')
    axes[1, 0].grid(axis='y', alpha=0.3)

# 4. Количество покупок в магазине
if 'NumStorePurchases' in available_features:
    store_by_cluster = df_with_clusters.groupby('SOM_Cluster')['NumStorePurchases'].mean()
    axes[1, 1].bar(store_by_cluster.index, store_by_cluster.values, color='purple')
    axes[1, 1].set_title('Среднее количество покупок в магазине', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('Кластер')
    axes[1, 1].set_ylabel('Среднее количество покупок')
    axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Тепловая карта характеристик кластеров
if 'Income' in available_features:
    # Выбираем ключевые признаки для визуализации
    key_features = ['Income', 'MntWines', 'MntMeatProducts', 'NumStorePurchases', 
                    'NumWebPurchases', 'NumCatalogPurchases', 'Recency']
    key_features = [f for f in key_features if f in available_features]
    
    cluster_heatmap = df_with_clusters.groupby('SOM_Cluster')[key_features].mean()
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(cluster_heatmap.T, annot=True, fmt='.1f', cmap='YlOrRd', 
                cbar_kws={'label': 'Среднее значение'})
    plt.title('Тепловая карта характеристик кластеров', fontsize=16, fontweight='bold')
    plt.xlabel('Кластер')
    plt.ylabel('Признак')
    plt.tight_layout()
    plt.show()


In [ ]:
# Интерпретация кластеров
print("\n" + "=" * 80)
print("ИНТЕРПРЕТАЦИЯ КЛАСТЕРОВ")
print("=" * 80)

for cluster_id in sorted(df_with_clusters['SOM_Cluster'].unique()):
    cluster_data = df_with_clusters[df_with_clusters['SOM_Cluster'] == cluster_id]
    print(f"\n--- Кластер {cluster_id} ---")
    print(f"Размер: {len(cluster_data)} клиентов ({len(cluster_data)/len(df_with_clusters)*100:.1f}%)")
    
    if 'Income' in available_features:
        print(f"Средний доход: ${cluster_data['Income'].mean():.2f}")
    if 'Age' in df_with_clusters.columns:
        print(f"Средний возраст: {cluster_data['Age'].mean():.1f} лет")
    if 'MntWines' in available_features:
        print(f"Средние расходы на вино: ${cluster_data['MntWines'].mean():.2f}")
    if 'NumStorePurchases' in available_features:
        print(f"Среднее количество покупок в магазине: {cluster_data['NumStorePurchases'].mean():.2f}")
    if 'Recency' in available_features:
        print(f"Средняя давность последней покупки: {cluster_data['Recency'].mean():.1f} дней")
    
    # Определяем тип кластера
    if 'Income' in available_features and 'MntWines' in available_features:
        avg_income = cluster_data['Income'].mean()
        avg_wine = cluster_data['MntWines'].mean()
        
        if avg_income > df_with_clusters['Income'].quantile(0.75):
            if avg_wine > df_with_clusters['MntWines'].quantile(0.75):
                cluster_type = "Премиум клиенты (высокий доход, высокие расходы)"
            else:
                cluster_type = "Богатые, но экономные"
        elif avg_income < df_with_clusters['Income'].quantile(0.25):
            cluster_type = "Бюджетные клиенты"
        else:
            cluster_type = "Средний сегмент"
        
        print(f"Тип: {cluster_type}")


## Задача 5: Сравнение с K-Means

Сравним результаты кластеризации SOM с методом K-Means.


In [ ]:
# Применяем K-Means напрямую к данным
kmeans_direct = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
kmeans_clusters = kmeans_direct.fit_predict(data_scaled)

df_with_clusters['KMeans_Cluster'] = kmeans_clusters

print("Распределение по кластерам K-Means:")
print(df_with_clusters['KMeans_Cluster'].value_counts().sort_index())


In [ ]:
# Визуализация сравнения кластеров
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Используем PCA для визуализации в 2D
pca = PCA(n_components=2, random_state=42)
data_2d = pca.fit_transform(data_scaled)

# SOM кластеры
scatter1 = axes[0].scatter(data_2d[:, 0], data_2d[:, 1], 
                          c=sample_clusters, cmap='tab10', alpha=0.6, s=20)
axes[0].set_title(f'SOM кластеризация (K={optimal_k})', fontsize=14, fontweight='bold')
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
axes[0].grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=axes[0], label='Кластер')

# K-Means кластеры
scatter2 = axes[1].scatter(data_2d[:, 0], data_2d[:, 1], 
                          c=kmeans_clusters, cmap='tab10', alpha=0.6, s=20)
axes[1].set_title(f'K-Means кластеризация (K={optimal_k})', fontsize=14, fontweight='bold')
axes[1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
axes[1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
axes[1].grid(True, alpha=0.3)
plt.colorbar(scatter2, ax=axes[1], label='Кластер')

plt.tight_layout()
plt.show()


In [ ]:
# Матрица сопряжённости для сравнения кластеров
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score

# Вычисляем метрики сходства
ari_score = adjusted_rand_score(sample_clusters, kmeans_clusters)
nmi_score = normalized_mutual_info_score(sample_clusters, kmeans_clusters)

print("Метрики сравнения кластеризаций:")
print(f"Adjusted Rand Index (ARI): {ari_score:.4f}")
print(f"Normalized Mutual Information (NMI): {nmi_score:.4f}")
print("\nИнтерпретация:")
print("- ARI = 1.0: идентичные кластеризации")
print("- ARI = 0.0: случайное совпадение")
print("- NMI = 1.0: полная взаимная информация")
print("- NMI = 0.0: нет взаимной информации")

# Матрица сопряжённости
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(sample_clusters, kmeans_clusters)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=[f'K-Means {i}' for i in range(optimal_k)],
            yticklabels=[f'SOM {i}' for i in range(optimal_k)])
plt.title('Матрица сопряжённости: SOM vs K-Means', fontsize=14, fontweight='bold')
plt.xlabel('K-Means кластеры')
plt.ylabel('SOM кластеры')
plt.tight_layout()
plt.show()


## Выводы (Conclusions)

### Основные результаты:

1. **SOM успешно обучена** на данных о клиентах с сеткой 10×10 нейронов
2. **U-Matrix визуализация** показала структуру кластеров в данных
3. **Выявлены кластеры клиентов** с различными характеристиками поведения
4. **Сравнение с K-Means** показало различия и сходства в подходах к кластеризации

### Преимущества SOM:

- **Топологическое сохранение**: похожие данные располагаются близко на карте
- **Визуализация**: U-Matrix позволяет визуально идентифицировать кластеры
- **Интерпретируемость**: можно анализировать характеристики каждого региона карты

### Отличия от K-Means:

- SOM сохраняет топологию данных, K-Means - нет
- SOM предоставляет визуальное представление структуры данных
- K-Means быстрее, но менее информативен для визуализации
